In [51]:
import numpy as np 
import pandas as pd
import torch

In [48]:
x = np.array([[1,2,3]])
print(x.shape)
x

(1, 3)


array([[1, 2, 3]])

In [49]:
y = np.array([[1],[2],[3]])
print(y.shape)
y

(3, 1)


array([[1],
       [2],
       [3]])

In [50]:
x + y

array([[2, 3, 4],
       [3, 4, 5],
       [4, 5, 6]])

In [53]:
torch.__version__

'2.3.1'

In [54]:
torch.cuda.is_available()

True

In [83]:
# create a 0D tensor (scalar) from a Python integer
tensor0d = torch.tensor(1)

# create a 1D tensor (vector) from a Python list
tensor1d = torch.tensor([1, 2, 3])

# create a 2D tensor from a nested Python list
tensor2d = torch.tensor([[1, 2, 3], [4, 5, 6]])

# create a 3D tensor from a nested Python list
tensor3d = torch.tensor([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])

In [84]:
tensor2d

tensor([[1, 2, 3],
        [4, 5, 6]])

In [86]:
tensor2d.T

tensor([[1, 4],
        [2, 5],
        [3, 6]])

In [87]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

loss.backward()

print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [96]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            # 2nd hidden layer
            torch.nn.Linear(30, 20),
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(20, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits

In [118]:
torch.manual_seed(123)
model = NeuralNetwork(50, 3)
print(model)
num_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad
)
print("Total number of trainable model parameters:", num_params)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)
Total number of trainable model parameters: 2213


In [119]:
model.layers[0].weight

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)

In [120]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [122]:
torch.manual_seed(123)

X = torch.rand((1, 50))
with torch.no_grad():
    out = model(X)
print(out)

tensor([[-0.1262,  0.1080, -0.1792]])


In [123]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])

y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])

y_test = torch.tensor([0, 1])

In [124]:
from torch.utils.data import Dataset


class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y

    def __len__(self):
        return self.labels.shape[0]


train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [131]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_ds,
    batch_size=2,
    shuffle=True,
    num_workers=0,
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,
    num_workers=0,
    drop_last=True
)

for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs=2,num_outputs=2)

#device = torch.device('cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # considered pytorch best practice
model.to(device)

optimizer = torch.optim.SGD(model.parameters(),lr=0.5)
epochs = 3

for epoch in range(epochs):

    model.train()

    for batch_idx, (features,labels) in enumerate(train_loader):
        features,labels = features.to(device), labels.to(device)

        logits = model(features)
        loss = F.cross_entropy(logits,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch: {epoch+1:03d}/{epochs:03d}" f" | Batch {batch_idx:03d}/{len(train_loader):03d}" f" | Train/Val Loss: {loss:.2f}")
      
    model.eval()

Epoch: 001/003 | Batch 000/002 | Train/Val Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train/Val Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train/Val Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train/Val Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train/Val Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train/Val Loss: 0.00


In [ ]:
# can make predictions
model.eval()
with torch.no_grad():
    outputs = model(X_train) # logits
print(outputs)

torch.set_printoptions(sci_mode=False) 
probs = torch.softmax(outputs,dim=1) # class probas
print(probs)

preds = torch.argmax(probs,dim=1) # convert probas to class preds
print(preds)
 
preds = torch.argmax(outputs,dim=1) # could also get preds direct from logits
print(preds)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])
tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])
tensor([0, 0, 0, 1, 1])
tensor([0, 0, 0, 1, 1])


In [ ]:
(torch.sum(preds == y_train) / len(y_train))*100 # acc

tensor(100.)

In [148]:
for key in model.state_dict(): 
    print(key)

layers.0.weight
layers.0.bias
layers.2.weight
layers.2.bias
layers.4.weight
layers.4.bias


In [149]:
torch.save(model.state_dict(),"model_test.pth")

In [150]:
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load("model_test.pth", weights_only=True))

<All keys matched successfully>